# **Etapa de Extração**

In [4]:
!pip install pandera

In [5]:
import pandas as pd
import pandera as pa

In [34]:
df = pd.read_csv('/content/ocorrencia_2010_2020.csv', sep = ';')
df

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,****,03/01/2010,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,03/01/2010,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,03/01/2010,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,****,04/01/2010,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,05/01/2010,19:25:00,0
...,...,...,...,...,...,...,...,...,...
5747,79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,29/12/2020,19:00:00,0
5748,79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,****,30/12/2020,18:30:00,0
5749,79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,30/12/2020,00:54:00,0
5750,79756,79756,INCIDENTE GRAVE,VICENTINA,MS,****,31/12/2020,09:00:00,0


In [7]:
df.dtypes

codigo_ocorrencia            int64
codigo_ocorrencia2           int64
ocorrencia_classificacao    object
ocorrencia_cidade           object
ocorrencia_uf               object
ocorrencia_aerodromo        object
ocorrencia_dia              object
ocorrencia_hora             object
total_recomendacoes          int64
dtype: object

In [8]:
#como dia é formato data, trocamos:
df = pd.read_csv('/content/ocorrencia_2010_2020.csv', sep = ';', parse_dates=['ocorrencia_dia'])
df

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,****,2010-03-01,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-03-01,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-03-01,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,****,2010-04-01,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-05-01,19:25:00,0
...,...,...,...,...,...,...,...,...,...
5747,79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00,0
5748,79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,****,2020-12-30,18:30:00,0
5749,79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,00:54:00,0
5750,79756,79756,INCIDENTE GRAVE,VICENTINA,MS,****,2020-12-31,09:00:00,0


In [9]:
df.head(5)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,****,2010-03-01,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-03-01,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-03-01,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,****,2010-04-01,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-05-01,19:25:00,0


In [10]:
df.ocorrencia_dia.dt.month

0        3
1        3
2        3
3        4
4        5
        ..
5747    12
5748    12
5749    12
5750    12
5751    12
Name: ocorrencia_dia, Length: 5752, dtype: int64

# **Etapa de Validação**

**Validação 1: consertando a data (dia-mes trocado)**

In [40]:
df = pd.read_csv('/content/ocorrencia_2010_2020.csv', sep = ';', parse_dates=['ocorrencia_dia'], dayfirst=True)
df

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,****,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,****,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
...,...,...,...,...,...,...,...,...,...
5747,79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00,0
5748,79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,****,2020-12-30,18:30:00,0
5749,79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,00:54:00,0
5750,79756,79756,INCIDENTE GRAVE,VICENTINA,MS,****,2020-12-31,09:00:00,0


**Essa etapa aqui serve para validar nosso dados, pois não basta só consertar a data da forma acima, pois existem casos que a data pode estar fora do padrão (exemplo colocar mês 15, sendo que pode até 12) e o código acima não faz a identificação disso.**

*biblioteca panderas para criar um schema.*

In [12]:
schema = pa.DataFrameSchema(
    columns = {
        'codigo_ocorrencia': pa.Column(pa.Int),
        'codigo_ocorrencia2': pa.Column(pa.Int),
        "ocorrencia_classificacao": pa.Column(pa.String),
        "ocorrencia_cidade": pa.Column(pa.String),
        "ocorrencia_uf": pa.Column(pa.String),
        "ocorrencia_aerodromo": pa.Column(pa.String),
        "ocorrencia_dia": pa.Column(pa.DateTime),
        "ocorrencia_hora": pa.Column(pa.String, nullable=True),
        "total_recomendacoes": pa.Column(pa.Int)
    }
)

*se passar, é pq tá tudo ok; senão, volta pra célula anterior e modifica algum parâmetro
i.e: nullable=True* *texto em itálico*

In [13]:
schema.validate(df)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,****,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,****,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
...,...,...,...,...,...,...,...,...,...
5747,79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00,0
5748,79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,****,2020-12-30,18:30:00,0
5749,79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,00:54:00,0
5750,79756,79756,INCIDENTE GRAVE,VICENTINA,MS,****,2020-12-31,09:00:00,0


*Fazendo uma validação do horário, coluna 'ocorrencia_hora', usando expressão regulares, limitando os valores para até 23:59:59*

In [14]:
schema = pa.DataFrameSchema(
    columns = {
        'codigo_ocorrencia': pa.Column(pa.Int),
        'codigo_ocorrencia2': pa.Column(pa.Int),
        "ocorrencia_classificacao": pa.Column(pa.String),
        "ocorrencia_cidade": pa.Column(pa.String),
        "ocorrencia_uf": pa.Column(pa.String),
        "ocorrencia_aerodromo": pa.Column(pa.String),
        "ocorrencia_dia": pa.Column(pa.DateTime),
        "ocorrencia_hora": pa.Column(pa.String, pa.Check.str_matches(r'^([0-1]?[0-9]|[2][0-3]):([0-5]?[0-9]):([0-5]?[0-9])?$'), nullable=True),
        "total_recomendacoes": pa.Column(pa.Int)
    }
)

In [15]:
schema.validate(df)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,****,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,****,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
...,...,...,...,...,...,...,...,...,...
5747,79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00,0
5748,79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,****,2020-12-30,18:30:00,0
5749,79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,00:54:00,0
5750,79756,79756,INCIDENTE GRAVE,VICENTINA,MS,****,2020-12-31,09:00:00,0


*Fazendo uma validação do horário, coluna 'ocorrencia_uf', usando expressão regulares, limitando a string para no min 2 e max 2*

In [16]:
schema = pa.DataFrameSchema(
    columns = {
        'codigo_ocorrencia': pa.Column(pa.Int),
        'codigo_ocorrencia2': pa.Column(pa.Int),
        "ocorrencia_classificacao": pa.Column(pa.String),
        "ocorrencia_cidade": pa.Column(pa.String),
        "ocorrencia_uf": pa.Column(pa.String, pa.Check.str_length(2,2)),
        "ocorrencia_aerodromo": pa.Column(pa.String),
        "ocorrencia_dia": pa.Column(pa.DateTime),
        "ocorrencia_hora": pa.Column(pa.String, pa.Check.str_matches(r'^([0-1]?[0-9]|[2][0-3]):([0-5]?[0-9]):([0-5]?[0-9])?$'), nullable=True),
        "total_recomendacoes": pa.Column(pa.Int)
    }
)

In [17]:
schema.validate(df)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,****,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,****,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
...,...,...,...,...,...,...,...,...,...
5747,79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00,0
5748,79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,****,2020-12-30,18:30:00,0
5749,79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,00:54:00,0
5750,79756,79756,INCIDENTE GRAVE,VICENTINA,MS,****,2020-12-31,09:00:00,0


*Caso estejamos usando uma alteração de colunas, onde as vezes ela é requerida,
as vezes não, usamos o required.*

In [18]:
schema = pa.DataFrameSchema(
    columns = {
        'coluna_variavel': pa.Column(pa.String, required=False),
        'codigo_ocorrencia': pa.Column(pa.Int),
        'codigo_ocorrencia2': pa.Column(pa.Int),
        "ocorrencia_classificacao": pa.Column(pa.String),
        "ocorrencia_cidade": pa.Column(pa.String),
        "ocorrencia_uf": pa.Column(pa.String, pa.Check.str_length(2,2)),
        "ocorrencia_aerodromo": pa.Column(pa.String),
        "ocorrencia_dia": pa.Column(pa.DateTime),
        "ocorrencia_hora": pa.Column(pa.String, pa.Check.str_matches(r'^([0-1]?[0-9]|[2][0-3]):([0-5]?[0-9]):([0-5]?[0-9])?$'), nullable=True),
        "total_recomendacoes": pa.Column(pa.Int)
    }
)

# **Etapa de Limpeza**

*Iremos usar uma função para certificar se realmente uma determinada coluna contém valores únicos em comparação as outras linhas*

In [21]:
df.codigo_ocorrencia.is_unique

True

*Como essa coluna contém valores únicos, iremos colocar o índice de cada linha desse df para os valores contidos nessa coluna*
*`não é necessário, porém ajuda em diminuir futuros erros`*

In [22]:
df.set_index('codigo_ocorrencia', inplace= True) #inplace para atualizar diretamente o df
df

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
codigo_ocorrencia,,,,,,,,
40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,****,2010-01-03,12:00:00,0
40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,****,2010-01-04,17:30:00,0
40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
...,...,...,...,...,...,...,...,...
79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00,0
79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,****,2020-12-30,18:30:00,0
79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,00:54:00,0


In [29]:
df.reset_index(drop=True, inplace=True)
df

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,INCIDENTE,RIO DE JANEIRO,RJ,2,2010-01-03,12:00:00,0
1,40349,INCIDENTE,BELÉM,PA,2,2010-01-03,11:05:00,0
2,40351,INCIDENTE,RIO DE JANEIRO,RJ,2,2010-01-03,03:00:00,0
3,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,2,2010-01-04,17:30:00,0
4,40324,INCIDENTE,PELOTAS,RS,2,2010-01-05,19:25:00,0
...,...,...,...,...,...,...,...,...
5747,79804,INCIDENTE,CAMPINAS,SP,2,2020-12-29,19:00:00,0
5748,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,2,2020-12-30,18:30:00,0
5749,79802,INCIDENTE,RIO DE JANEIRO,RJ,2,2020-12-30,00:54:00,0
5750,79756,INCIDENTE GRAVE,VICENTINA,MS,2,2020-12-31,09:00:00,0


*Realizando backup em uma coluna*

In [23]:
df['ocorrencia_aerodromo_backup'] = df.ocorrencia_aerodromo
df

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_aerodromo_backup
codigo_ocorrencia,,,,,,,,,
40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,****,2010-01-03,12:00:00,0,****
40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0,SBBE
40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0,SBRJ
39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,****,2010-01-04,17:30:00,0,****
40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0,SBPK
...,...,...,...,...,...,...,...,...,...
79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00,0,SBKP
79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,****,2020-12-30,18:30:00,0,****
79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,00:54:00,0,SBGL


*Excluindo uma coluna toda*

In [26]:
df.drop('ocorrencia_aerodromo_backup', axis = 1, inplace = True)
df

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
codigo_ocorrencia,,,,,,,,
40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,****,2010-01-03,12:00:00,0
40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,****,2010-01-04,17:30:00,0
40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
...,...,...,...,...,...,...,...,...
79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00,0
79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,****,2020-12-30,18:30:00,0
79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,00:54:00,0


*Modificando uma coluna toda*

In [27]:
df['ocorrencia_aerodromo'] = 2
df

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
codigo_ocorrencia,,,,,,,,
40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,2,2010-01-03,12:00:00,0
40349,40349,INCIDENTE,BELÉM,PA,2,2010-01-03,11:05:00,0
40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,2,2010-01-03,03:00:00,0
39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,2,2010-01-04,17:30:00,0
40324,40324,INCIDENTE,PELOTAS,RS,2,2010-01-05,19:25:00,0
...,...,...,...,...,...,...,...,...
79804,79804,INCIDENTE,CAMPINAS,SP,2,2020-12-29,19:00:00,0
79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,2,2020-12-30,18:30:00,0
79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,2,2020-12-30,00:54:00,0


*Modificando uma linha toda*

In [31]:
df.loc[1] = 'oi'
df

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,1
0,40211,INCIDENTE,RIO DE JANEIRO,RJ,2,2010-01-03 00:00:00,12:00:00,0,oi
1,oi,oi,oi,oi,oi,oi,oi,oi,oi
2,40351,INCIDENTE,RIO DE JANEIRO,RJ,2,2010-01-03 00:00:00,03:00:00,0,oi
3,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,2,2010-01-04 00:00:00,17:30:00,0,oi
4,40324,INCIDENTE,PELOTAS,RS,2,2010-01-05 00:00:00,19:25:00,0,oi
...,...,...,...,...,...,...,...,...,...
5747,79804,INCIDENTE,CAMPINAS,SP,2,2020-12-29 00:00:00,19:00:00,0,oi
5748,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,2,2020-12-30 00:00:00,18:30:00,0,oi
5749,79802,INCIDENTE,RIO DE JANEIRO,RJ,2,2020-12-30 00:00:00,00:54:00,0,oi
5750,79756,INCIDENTE GRAVE,VICENTINA,MS,2,2020-12-31 00:00:00,09:00:00,0,oi


*Modificando um valor específico no dataframe*

In [32]:
df.loc[1, 'codigo_ocorrencia2'] = 'ola'
df

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,1
0,40211,INCIDENTE,RIO DE JANEIRO,RJ,2,2010-01-03 00:00:00,12:00:00,0,oi
1,ola,oi,oi,oi,oi,oi,oi,oi,oi
2,40351,INCIDENTE,RIO DE JANEIRO,RJ,2,2010-01-03 00:00:00,03:00:00,0,oi
3,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,2,2010-01-04 00:00:00,17:30:00,0,oi
4,40324,INCIDENTE,PELOTAS,RS,2,2010-01-05 00:00:00,19:25:00,0,oi
...,...,...,...,...,...,...,...,...,...
5747,79804,INCIDENTE,CAMPINAS,SP,2,2020-12-29 00:00:00,19:00:00,0,oi
5748,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,2,2020-12-30 00:00:00,18:30:00,0,oi
5749,79802,INCIDENTE,RIO DE JANEIRO,RJ,2,2020-12-30 00:00:00,00:54:00,0,oi
5750,79756,INCIDENTE GRAVE,VICENTINA,MS,2,2020-12-31 00:00:00,09:00:00,0,oi


**fiz um reset (compilei o df novamente) a partir daqui**

Fazendo uma varredura dos valores incorretos no .csv, temos:

ocorrencia_uf
**

ocorrencia_aerodromo
###
####
\***
\****

ocorrencia_hora
NULL


In [41]:
df.replace(['**', '###!', '####', '***', '****'], pd.NA, inplace=True)
df

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,<NA>,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,<NA>,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
...,...,...,...,...,...,...,...,...,...
5747,79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00,0
5748,79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,<NA>,2020-12-30,18:30:00,0
5749,79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,00:54:00,0
5750,79756,79756,INCIDENTE GRAVE,VICENTINA,MS,<NA>,2020-12-31,09:00:00,0


In [42]:
df.tail()

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
5747,79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00,0
5748,79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,<NA>,2020-12-30,18:30:00,0
5749,79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,00:54:00,0
5750,79756,79756,INCIDENTE GRAVE,VICENTINA,MS,<NA>,2020-12-31,09:00:00,0
5751,79844,79844,INCIDENTE,RIO DE JANEIRO,RJ,SBJR,2020-12-31,13:24:00,0
